In [3]:
#ide na git
pip install TensorFlow

SyntaxError: invalid syntax (<ipython-input-3-fecc1bf87a27>, line 2)

In [4]:
pip install Keras

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pillow


Note: you may need to restart the kernel to use updated packages.


In [3]:

pip install NumPy


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install jupyterlab

In [5]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
from keras.applications.xception import Xception #to get pre-trained model Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer #for text tokenization
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense#Keras to build our CNN and LSTM
from keras.layers import LSTM, Embedding, Dropout
from tqdm import tqdm_notebook as tqdm #to check loop progress
tqdm().pandas()

<ipython-input-5-b96bcb645da8>:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


0it [00:00, ?it/s]

In [16]:
def load_file(file_name):
    file = open(file_name, 'r')
    text = file.read()
    file.close()
    return text

def img_capt(file_name):
    tekst = load_file(file_name)
    words = tekst.split('\n')
    descriptions ={}
    for word in words[:-1]:
        #print(word)
        img, word = word.split(':')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [ word ]
        else:
            descriptions[img[:-2]].append(word)
    return descriptions

def txt_vocab(descriptions):
  # To build vocab of all unique words
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab



file_name = "ImageNotes1April.txt" #inace bez 1April

descriptions = img_capt(file_name)
print("Length of descriptions =" ,len(descriptions))

#to build vocabulary
vocabulary = txt_vocab(descriptions)
print("Length of vocabulary = ", len(vocabulary))

Length of descriptions = 826
Length of vocabulary =  3999


In [18]:
def extract_features(directory):
    model = Xception( include_top=False, pooling='avg' )
    features = {}
    for pic in tqdm(os.listdir(directory)):
        file = directory + "/" + pic
        picture = Image.open(file)
        picture = picture.resize((299,299))
        picture = np.expand_dims(picture, axis=0)
          #image = preprocess_input(image)
        picture = picture/127.5
        picture = picture - 1.0
        if picture.shape[0] != 1 or picture.shape[1] != 299 or picture.shape[2] != 299 or picture.shape[3] != 3:
            continue
        else:
            
            feature = model.predict(picture)
            features[pic] = feature
    return features
#2048 feature vector
dataset_images =  "obradjeno"#test_images"# "C:/soft/bb/MusicGenerator/flick"
features = extract_features(dataset_images)
dump(features, open("features.p","wb"))
#to directly load the features from the pickle file.
features = load(open("features.p","rb"))

<ipython-input-18-1c75a9c744d4>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for pic in tqdm(os.listdir(directory)):


0it [00:00, ?it/s]

In [19]:
def load_images(filename):
    text = load_file(filename)
    rows = text.split("\n")
    image_names = []
    
    for row in rows:
        image_names.append(row.split(":")[0].strip())
        
    return image_names

def load_clean_descriptions(filename, photos):
  #loading clean_descriptions
    tekst = load_file(filename)
    descriptions = {}
    for line in tekst.split("\n"):
        words = line.split(":")
        if len(words)<=1 :
            continue
        img, img_desc = words[0].strip(), words[1].split(" ")
        if img in photos:
            if img not in descriptions:
                descriptions[img] = []
            desc = ' ' + " ".join(img_desc) + ' '
            descriptions[img].append(desc)
    return descriptions

def load_features(photos):
 
    all_features = load(open("features.p","rb"))
    
    features = {}
    #print(all_features)
    for k in photos:
        try:
            features[(k.split(".png")[0]+".jpg")] = all_features[(k.split(".png")[0]+".jpg")]
        except:
            print(k)
            continue
    return features

train_images = load_images("imageNotes1April.txt")
train_descriptions = load_clean_descriptions("imageNotes1April.txt", train_images)
train_features = load_features(train_images)


10-Yard Fight - square7.png
1943 - square3.png
1999 - square2.png
3D Battles of Worldrunner, The - square0.png
3D Galax - square0.png
3D StarFighter - square11.png
4 Nin Shogi (Japan) - square6.png
4-in-1 Fun Pak - square6.png
4d Boxing - square2.png
4th & Inches - square0.png
7 Gates Of Jambala - square0.png
7 Gates Of Jambala - square3.png
7th Saga - square10.png
7th Saga - square16.png
8 Eyes - square2.png
9 Lives - square12.png
90 Minutes_ European Prime Goal - square6.png
A Boy and His Blob - square0.png
A Bug's Life - square0.png
A Nightmare on Elm Street - square10.png
A Prehistoric Tale - square0.png
Abadox - square8.png
Ace Combat - square0.png
Ace Combat 04_ Shattered Skies - square0.png
Ace Combat 2 - square2.png
Action 52 - square1.png
Action Biker - square2.png
ActRaiser - square0.png
ActRaiser 2 - square15.png
Addams Family, The_ Pugsley's Scavenger Hunt - square1.png
Advance Guardian Heroes - square0.png
Advance Wars - square4.png
Advance Wars 2_ Black Hole Rising - squa

In [11]:
from keras.preprocessing.text import Tokenizer

def dict_to_list(descriptions): 
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open('tokenizer.p', 'wb'))
vocab_size = len(tokenizer.word_index) + 1
vocab_size 

def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

max_length = max_length(descriptions)
#print(descriptions)
max_length #Max_length  6

6

In [12]:

def data_generator(train_descriptions, features, tokenizer, max_length):
    while 1:
        for key, description_list in train_descriptions.items():
            feature = features[key.split(".png")[0]+".jpg"][0]
            inp_image, inp_seqence, op_word = create_sequences(tokenizer, max_length, description_list, feature)
            yield [inp_image, inp_seqence], op_word

def create_sequences(tokenizer, max_length, desc_list, feature):
    x_1, x_2, y = list(), list(), list()
    for desc in desc_list:
        seqence= tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seqence)):
            in_seqence, out_seqence = seqence[:i], seqence[i]
            in_seqence = pad_sequences([in_seqence], maxlen=max_length)[0]
            out_seqence = to_categorical([out_seqence], num_classes=vocab_size)[0]
            x_1.append(feature)
            x_2.append(in_seqence)
            y.append(out_seqence)
    return np.array(x_1), np.array(x_2), np.array(y)

[a,b],c = next(data_generator(train_descriptions, features, tokenizer, max_length))
a.shape, b.shape, c.shape
#((24, 2048), (24, 6), (24, 639))

((24, 2048), (24, 6), (24, 639))

In [13]:
from keras.utils import plot_model

def define_model(vocab_size, max_length):
  
    inputs1 = Input(shape=(2048,)) #odozgo je broj
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
 
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
 
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
 
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
  
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [14]:
# train our model
print('Dataset: ', len(train_images))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)
model = define_model(vocab_size, max_length)
epochs = 10
steps = len(train_descriptions)
print(steps)
# creating a directory named models to save our models
#os.mkdir("models")
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=10, steps_per_epoch= steps, verbose=1)
    model.save("models/model_" + str(i) + ".h5")

Dataset:  852
Descriptions: train= 826
Photos: train= 826
Vocabulary Size: 639
Description Length:  6
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 2048)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 6, 256)       163584      input_5[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 2048)         0           input_4[0][0]      

KeyboardInterrupt: 

In [73]:
pip install pydot